In [1]:
import numpy as np

In [2]:
from pycuda import gpuarray

In [2]:
from time import time

In [4]:
import pycuda.driver as cuda
import pycuda.tools
import pycuda.autoinit

In [5]:
dim = 1000
rnd = np.random.RandomState(0)
a = rnd.rand(dim, dim).astype(np.float32)
b = rnd.rand(dim, dim).astype(np.float32)
a_gpu = gpuarray.to_gpu(a)
b_gpu = gpuarray.to_gpu(b)

# start = time.time()
# rescpu = np.dot(a, b)
# print 'CPU:', time.time() - start

start = time()
resgpu = linalg.dot(a_gpu, b_gpu)
print ('GPU:', time() - start)


# resgpu = resgpu.get()
# print np.allclose(rescpu, resgpu)
# print np.allclose(resgpu, rescpu)

NameError: name 'linalg' is not defined

In [3]:
import pycuda.driver as cuda
import pycuda.tools
import pycuda.autoinit
import numpy
import numpy.linalg as la
from pycuda.compiler import SourceModule

mod = SourceModule("""
__global__ void multiply_them(float *dest, float *a, float *b)
{
    const int i = threadIdx.x;
    dest[i] = a[i] * b[i];
}
""")

multiply_them = mod.get_function("multiply_them")

a = numpy.random.randn(1000000).astype(numpy.float32)
b = numpy.random.randn(1000000).astype(numpy.float32)

dest = numpy.zeros_like(a)
start = time()
multiply_them(
        cuda.Out(dest), cuda.In(a), cuda.In(b),
        block=(400,1,1))
print (time() - start)
start = time()
c = a * b
print (time() - start)

print (dest-a*b)

0.108999967575
0.0149998664856
[ 0.          0.          0.         ..., -0.81707287  0.41746762
  0.82374561]


In [4]:
%%timeit
multiply_them(
        cuda.Out(dest), cuda.In(a), cuda.In(b),
        block=(400,1,1))

100 loops, best of 3: 6.88 ms per loop


In [27]:
dest

array([-0.46173283, -0.50773519,  0.07551581, ...,  0.        ,
        0.        ,  0.        ], dtype=float32)

In [23]:
%%timeit
a*b

The slowest run took 7.37 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 17.5 µs per loop


In [20]:
new = a*b

In [21]:
new.shape

(40000L,)

In [33]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [68]:
mem_free, mem_total = cuda.mem_get_info()

In [69]:
float(mem_free)/float(mem_total)

0.7976409909315407

In [2]:
import numpy
a = numpy.random.randn(4,4)

In [35]:
a = a.astype(numpy.float32)

In [36]:
a_gpu = cuda.mem_alloc(a.nbytes)

In [8]:
mod = SourceModule("""
#include <stdio.h>
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    printf("I am %d.%d\\n", threadIdx.x, threadIdx.y);
    a[idx] *= 2;
  }
  """)

In [41]:
func = mod.get_function("doublify")

In [42]:
func(a_gpu, block=(4,4,1))

In [11]:
mod = SourceModule("""
    #include <stdio.h>

    __global__ void say_hi()
    {
      printf("I am %d.%d\\n", threadIdx.x, threadIdx.y);
    }
    """)

func = mod.get_function("say_hi")
func(block=(4,4,1))

In [68]:
X = np.loadtxt("table.txt")

In [78]:
X.astype(np.int32)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [47]:
sigRef = np.array([2, 7, 3, 12, 10, 8, 11, 6, 1, -3, -2, 5, 10, 4, -2, -8, 0, 6, 3, 2, 10, 15, 12, 7])
sigBase = np.array([7, 10, 7, 1, 1, 1, -3, 0, 6, 9, 4, -2])

In [48]:
sigRef.shape[0]

24L

In [49]:
dissMatr = np.zeros((sigRef.shape[0], sigBase.shape[0]))
for i in xrange(sigRef.shape[0]):
    for j in xrange(sigBase.shape[0]):
        dissMatr[i,j] = abs(sigRef[i] - sigBase[j])

In [51]:
X = dissMatr

In [75]:
pnt = 2
m, n = X.shape
D = np.zeros((m+1, n+1))
D[0,0] = 0
D[0,:] = np.inf
D[:,0] = np.inf
D[1:(m+1), 1:(n+1)] = X
D_gpu = gpuarray.to_gpu(D)
phi = np.zeros((m, n))
print D
start = time()
for i in xrange(0,m):  # почему с 0? тут nan
    for j in xrange(1,n):
        tmp = [D[i, j], D[i, j + 1] + pnt, D[i + 1, j] + pnt]
        tb = np.argmin(tmp)
        if i == 1:
            print tmp
            print tb
        dmin = tmp[tb]
        D[i + 1, j + 1] = D[i + 1, j + 1] + dmin
        phi[i, j] = tb+1
print time()-start
print phi
j = n
i = np.argmin(D[1:, n])
print "i:", i
res = D[1:, n][i]
print "res:", res
path = np.array([[i],[j]])
print path
start = time()
print i,j
while j > 1 and i > 1:
    tb = phi[i, j-1]
    if tb == 1:
        i -= 1
        j -= 1
    elif tb == 2:
        i -= 1
    elif tb == 3:
        j -= 1
    else:
        print i,j
        raise Exception("wtf?")

    path = np.insert(path, 0, [i,j], axis=1)

D = D[1:(m + 1), 1:(n + 1)]
print time() - start
print path

[[ inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf]
 [ inf   5.   8.   5.   1.   1.   1.   5.   2.   4.   7.   2.   4.]
 [ inf   0.   3.   0.   6.   6.   6.  10.   7.   1.   2.   3.   9.]
 [ inf   4.   7.   4.   2.   2.   2.   6.   3.   3.   6.   1.   5.]
 [ inf   5.   2.   5.  11.  11.  11.  15.  12.   6.   3.   8.  14.]
 [ inf   3.   0.   3.   9.   9.   9.  13.  10.   4.   1.   6.  12.]
 [ inf   1.   2.   1.   7.   7.   7.  11.   8.   2.   1.   4.  10.]
 [ inf   4.   1.   4.  10.  10.  10.  14.  11.   5.   2.   7.  13.]
 [ inf   1.   4.   1.   5.   5.   5.   9.   6.   0.   3.   2.   8.]
 [ inf   6.   9.   6.   0.   0.   0.   4.   1.   5.   8.   3.   3.]
 [ inf  10.  13.  10.   4.   4.   4.   0.   3.   9.  12.   7.   1.]
 [ inf   9.  12.   9.   3.   3.   3.   1.   2.   8.  11.   6.   0.]
 [ inf   2.   5.   2.   4.   4.   4.   8.   5.   1.   4.   1.   7.]
 [ inf   3.   0.   3.   9.   9.   9.  13.  10.   4.   1.   6.  12.]
 [ inf   3.   6.   3.   3.   3.   3.   7.   4.  

In [67]:
# переделка

pnt = 2
m, n = X.shape
D = np.zeros((m+1, n+1))
D[0,0] = 0
D[0,:] = np.inf
D[:,0] = np.inf
D[1:(m+1), 1:(n+1)] = X
D_gpu = gpuarray.to_gpu(D)
phi = np.zeros((m, n))
print D
start = time()
for i in xrange(0,m):  # почему с 0? тут nan
    for j in xrange(1,n):
        tmp = [D[i, j], D[i, j + 1] + pnt, D[i + 1, j] + pnt]
        print tmp
        tb = np.argmin(tmp)
        dmin = tmp[tb]
        D[i + 1, j + 1] = D[i + 1, j + 1] + dmin
        phi[i, j] = tb+1
print time()-start
print phi
j = n
i = np.argmin(D[1:, n])
print "i:", i
res = D[1:, n][i]
print "res:", res
path = np.array([[i],[j]])
print path
start = time()
print i,j
while j > 1:
    tb = phi[i, j-1]
    if tb == 1:
        i -= 1
        j -= 1
    elif tb == 2:
        i -= 1
    elif tb == 3:
        j -= 1
    else:
        print i,j
        raise Exception("wtf?")

    path = np.insert(path, 0, [i,j], axis=1)

D = D[1:(m + 1), 1:(n + 1)]
print time() - start
print path

[[ inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf]
 [ inf   5.   8.   5.   1.   1.   1.   5.   2.   4.   7.   2.   4.]
 [ inf   0.   3.   0.   6.   6.   6.  10.   7.   1.   2.   3.   9.]
 [ inf   4.   7.   4.   2.   2.   2.   6.   3.   3.   6.   1.   5.]
 [ inf   5.   2.   5.  11.  11.  11.  15.  12.   6.   3.   8.  14.]
 [ inf   3.   0.   3.   9.   9.   9.  13.  10.   4.   1.   6.  12.]
 [ inf   1.   2.   1.   7.   7.   7.  11.   8.   2.   1.   4.  10.]
 [ inf   4.   1.   4.  10.  10.  10.  14.  11.   5.   2.   7.  13.]
 [ inf   1.   4.   1.   5.   5.   5.   9.   6.   0.   3.   2.   8.]
 [ inf   6.   9.   6.   0.   0.   0.   4.   1.   5.   8.   3.   3.]
 [ inf  10.  13.  10.   4.   4.   4.   0.   3.   9.  12.   7.   1.]
 [ inf   9.  12.   9.   3.   3.   3.   1.   2.   8.  11.   6.   0.]
 [ inf   2.   5.   2.   4.   4.   4.   8.   5.   1.   4.   1.   7.]
 [ inf   3.   0.   3.   9.   9.   9.  13.  10.   4.   1.   6.  12.]
 [ inf   3.   6.   3.   3.   3.   3.   7.   4.  

In [80]:
np.savetxt("path.txt", path)

In [43]:
path

array([[-8, -7, -7, -7, -6, -5, -5, -5, -5, -4, -3, -2, -1,  0],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 10, 11, 12]], dtype=int64)

In [41]:
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

In [15]:
def meminfo(kernel):
    shared=kernel.shared_size_bytes
    regs=kernel.num_regs
    local=kernel.local_size_bytes
    const=kernel.const_size_bytes
    mbpt=kernel.max_threads_per_block
    print("""=MEM=\nLocal:%d,\nShared:%d,\nRegisters:%d,\nConst:%d,\nMax Threads/B:%d"""%(local,shared,regs,const,mbpt))

In [16]:
kernel=mod.get_function("multiply_them")
meminfo(kernel)

=MEM=
Local:0,
Shared:0,
Registers:10,
Const:0,
Max Threads/B:1024


In [31]:
mod.get_function("multiply_them").max_threads_per_block

1024